In [4]:
import torch

class MLP(torch.nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用MLP的父类Module的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数params（稍后将介绍）
        super().__init__()
        self.hidden = torch.nn.Linear(20, 256)  # 隐藏层
        self.out = torch.nn.Linear(256, 10)  # 输出层

    # 定义模型的前向传播，即如何根据输入X返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for idx, module in enumerate(args):
            # 这里，module是Module子类的一个实例。我们把它保存在'Module'类的成员
            # 变量_modules中。_module的类型是OrderedDict
            self._modules[str(idx)] = module

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X
    
class FixedHiddenMLP(nn.Module):
    '''
        在这个FixedHiddenMLP模型中，我们实现了一个隐藏层， 其权重（self.rand_weight）在实例化时被随机初始化，之后为常量。 这个权重不是一个模型参数，因此它永远不会被反向传播更新。 然后，神经网络将这个固定层的输出通过一个全连接层。

        注意，在返回输出之前，模型做了一些不寻常的事情： 它运行了一个while循环，在L1范数大于1的条件下， 将输出向量除以2，直到它满足条件为止。 
        最后，模型返回了X中所有项的和。 注意，此操作可能不会常用于在任何实际任务中， 我们只展示如何将任意代码集成到神经网络计算的流程中。
    '''
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及relu和mm函数
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

class NestMLP(nn.Module):
    '''
        我们可以混合搭配各种组合块的方法。 
    '''
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))
 
# 定义模型 三层卷积 一层全连接
class xavierNet(torch.nn.Module):
    def __init__(self):
        super(xavierNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 1, 3)
        print('random init:', self.conv1.weight)
        '''
            xavier 初始化方法中服从均匀分布 U(−a,a) ，分布的参数 a = gain * sqrt(6/fan_in+fan_out)，
            这里有一个 gain，增益的大小是依据激活函数类型来设定,该初始化方法，也称为 Glorot initialization
        '''
        torch.nn.init.xavier_uniform_(self.conv1.weight, gain=1)
        print('xavier_uniform_:', self.conv1.weight)
        '''
            xavier 初始化方法中服从正态分布，
            mean=0,std = gain * sqrt(2/fan_in + fan_out)
        '''
        torch.nn.init.xavier_normal_(self.conv1.weight, gain=1)
        print('xavier_uniform_:', self.conv1.weight)
        
# 定义模型 三层卷积 一层全连接
class heNet(torch.nn.Module):
    def __init__(self):
        super(heNet, self).__init__()
        self.conv1 = torch.nn.Conv2d(1, 1, 3)
        print('random init:', self.conv1.weight)
        '''
            此为均匀分布，U~(-bound,bound)，bound=sqrt(6/(1+a^2)*fan_in)
            其中，a为激活函数的负半轴的斜率，relu是0
            mode- 可选为fan_in或fan_out,fan_in使用正向传播，方差一致；fan_out使反向传播时，方差一致nonlinearity-可选relu和leaky_relu，默认值为：leaky_relu
        '''
        torch.nn.init.kaiming_uniform(self.conv1.weight, mode='fan_in', nonlinearity='relu')
        print('kaiming_uniform:', self.conv1.weight)
        '''
            此为0均值的正态分布，N~(0,std),
            其中std=sqrt(2/(1+a^2)*fan_in)其中，a为激活函数的负半轴的斜率，relu是0
            mode- 可选为fan_in或fan_out,fan_in使正向传播时，方差一致；fan_out使反向传播时，方差一致。nonlinearity-可选relu和leaky_relu，默认值为leaky_relu.
        '''
        torch.nn.init.kaiming_normal_(self.conv1.weight, mode='fan_in', nonlinearity='relu')
        print('kaiming_normal_:', self.conv1.weight)
 
# 均匀分布初始化 使值服从均匀分布U(a,b)
# torch.nn.init.uniform_(tensor,a=0,b=1)

# 正态分布初始化，使值服从正态分布N(mean,std)，默认值为0，1
# torch.nn.init.normal_(tensor,mean=0,std=1)

# 常数初始化，值为常数val nn.init.constant_(w,0.3)
# torch.nn.init.constant_(tensor,val)

# 单位矩阵初始化，将二维tensor初始化为单位矩阵
# torch.nn.init.eye_(tensor)

# 正交初始化，使得tensor是正交的
# torch.nn.init.orthogonal_(tensor,gain=1)

# 稀疏初始化，从正态分布N~(0,std)中进行稀疏化，使每一个column有一部分为0,sparsity-每一个column稀疏的比例，即为0的比例
# torch.nn.init.sparse_(tensor,sparsity,std=0.01)
# **nn.init.sparse_(w,sparsity=0.1)

# 计算增益
# torch.nn.init.calculate_gain(nonlinearity,param=None)



 
if __name__ == '__main__':
    net = MLP()
    net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
    net = FixedHiddenMLP()
    chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())

    xavierNet = xavierNet()
    heNet = heNet()

random init: Parameter containing:
tensor([[[[-0.0444,  0.1668,  0.0907],
          [-0.2761,  0.2433,  0.3265],
          [ 0.0993,  0.0607,  0.2189]]]], requires_grad=True)
xavier_uniform_: Parameter containing:
tensor([[[[ 0.4527, -0.4683, -0.4501],
          [ 0.2653,  0.2067,  0.1232],
          [-0.1232,  0.4641, -0.1927]]]], requires_grad=True)
xavier_uniform_: Parameter containing:
tensor([[[[ 0.0771,  0.7910,  0.0256],
          [-0.1816, -0.0185,  0.3900],
          [-0.7934,  0.0983,  0.0962]]]], requires_grad=True)
random init: Parameter containing:
tensor([[[[-0.2728,  0.0664,  0.2419],
          [ 0.3078,  0.3288,  0.0764],
          [-0.1838,  0.1033, -0.1104]]]], requires_grad=True)
kaiming_uniform: Parameter containing:
tensor([[[[ 0.5699,  0.8126, -0.4218],
          [-0.0843, -0.6801,  0.0921],
          [ 0.5388, -0.1393,  0.0163]]]], requires_grad=True)
kaiming_normal_: Parameter containing:
tensor([[[[ 0.0801,  0.2345, -0.2808],
          [-0.1700,  0.2664, -0.766

/tmp/ipykernel_2332222/3648542355.py:47: UserWarning: nn.init.kaiming_uniform is now deprecated in favor of nn.init.kaiming_uniform_.
  torch.nn.init.kaiming_uniform(self.conv1.weight, mode='fan_in', nonlinearity='relu')
